# Module 5 Lesson 5

In [5]:
#import libraries
import numpy as np
from matplotlib import pyplot
from helper import l2_norm, poisson_2d_jacobi, poisson_solution
import numba
from numba import jit

%matplotlib inline

In [6]:
#Parameter establishment
nx, ny = 101, 101 #number of points in x/y
xmin, xmax = 0.0, 1.0 #limits in x
ymin, ymax = -0.5, 0.5 #limits in y
Lx = xmax - xmin #length x
Ly = ymax - ymin #length y
dx = Lx / (nx - 1) #delta in x
dy = Ly / (ny - 1) #delta in y

#create mapping
x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y) #create mapping in X/Y

#source term
b = (-2 * (np.pi / Lx) * (np.pi / Ly) * 
     np.sin(np.pi * X / Lx) * 
     np.cos(np.pi * Y / Ly))

#Establish initial condition
p0 = np.zeros((ny, nx))
#Compute exact
pe = poisson_solution(x, y, Lx, Ly)

In [54]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    """Solves the 2D Poisson equiation on a uniform grid
    utilizing the method of steepest descent"""
    
    def A(p):
        """Apply laplacian operator to p - given that dx = dy"""
        return (-4.0 * p[1:-1, 1:-1] + 
                p[1:-1, :-2] + p[1:-1, 2:] + 
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        #compute residual
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        #compute laplacian of residual
        Ar[1:-1, 1:-1] = A(r)
        #compute step size
        alpha = np.sum(r * r) / np.sum(r * Ar)
        #update solution
        p = pk + alpha * r
        #dirichlet BC enforced by not changing boundary values
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [65]:
p_sd, ite_sd, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, 
                                                     maxiter=20000,
                                                     rtol=1e-10)

In [59]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, 
                                 maxiter=20000,
                                 rtol=1e-6):
    """Solve the poisson 2d problm by utilizing
    the conjugate gradient method"""
    def A(p):
        """Solves the laplacian equation in 2d using the CD method"""
        return (-4 * p[1:-1, 1:-1] + p[1:-1, 2:] + p[1:-1, :-2]
               + p[2:, 1:-1] + p[:-2, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    rk = numpy.zeros_like(p)
    Ad = numpy.zeros_like(p)
    conv = []
    ites = 0
    diff = rtol + 1
    #compute initial residual
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ites < maxiter:
        pk = p.copy()
        rk = r.copy()
        Ad[1:-1, 1:-1] = A(d)
        alpha = np.sum(r * r) / np.sum(d * Ad)
        p = pk + alpha * d
        r = rk - alpha * Ad
        beta = np.sum(r * r) / np.sum(rk * rk)
        d = r + beta * d
        diff = l2_norm(p, pk)
        conv.append(diff)
        ites += 1
    return p, ites, conv

In [60]:
p_cg, ite_cg, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy, 
                                                     maxiter=20000,
                                                     rtol=1e-10)

In [71]:
p_j, ite_j, conv_j = poisson_2d_jacobi(p0, b, dx, dy, 
                                       maxiter = 40000, rtol=1e-10)

In [72]:
print(conv_cg[-1])
print(conv_sd[-1])
print(conv_j[-1])

1.3087029879310584e-16
1.3307695446303778e-16
9.997923503623598e-11


In [73]:
print(ite_cg)
print(ite_sd)
print(ite_j)

2
2
31227


In [75]:
b2 = (np.sin(np.pi * X / Lx) * 
     np.cos(np.pi * Y / Ly) + 
     np.sin(6.0 * np.pi * X / Lx) * 
     np.sin(6.0 * np.pi * Y / Ly))
p_sd_2, ite_sd_2, conv_sd_2 = poisson_2d_steepest_descent(p0, b2, dx, dy, maxiter = 40000, rtol = 1e-10)
p_cg_2, ite_cg_2, conv_cg_2 = poisson_2d_conjugate_gradient(p0, b2, dx, dy, maxiter = 40000, rtol = 1e-10)
p_j_2, ite_j_2, conv_j_2 = poisson_2d_jacobi(p0, b2, dx, dy, maxiter = 40000, rtol = 1e-10)

In [76]:
print(conv_cg_2[-1])
print(conv_sd_2[-1])
print(conv_j_2[-1])

2.4657658526086296e-14
9.998128475296845e-11
9.998980840555589e-11


In [77]:
print(ite_cg_2)
print(ite_sd_2)
print(ite_j_2)

3
27059
31226
